In [1]:
# !python -m spacy download en_core_web_sm
# ! pip install swifter
# ! pip install spacytextblob
# ! pip install -U profanity-filter
# ! pip uninstall spacy
# ! pip install flair
# ! python -m pip install "dask[dataframe]"   


In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from spacytextblob.spacytextblob import SpacyTextBlob
from profanity_filter import ProfanityFilter
import dask.dataframe as dd
import nltk,spacy,swifter

In [4]:
# nlp_spacy = spacy.load('en_core_web_sm')
nlp_spacy = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])
# nlp_spacy.add_pipe('spacytextblob')
# nlp_spacy.add_pipe('profanity_filter')
# nltk.download('stopwords')
stops = set(stopwords.words('english'))

In [16]:
%%time
df = pd.read_csv(r"C:\Users\prasr\Documents\Northeastern\IDMP\Project\sarcasm_detection\data\raw\train-balanced-sarcasm.csv")
df.head(5)

Wall time: 3.3 s


,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


# 1. Looking into the parent comments

## 1.1 Clean the parent comments

In [5]:
def normalize_text(comment):
    nlp_doc = nlp_spacy(comment.lower())
    lemmatized_out_list = []
    for token in nlp_doc:
        if token not in stops and (token.is_alpha or token.like_num or token.like_url):
            lemmatized_out_list.append(token.lemma_.strip())
    out = " ".join(lemmatized_out_list)
    return out

In [19]:
%%time
df['cleaned_parent_comment'] = df.swifter.apply(lambda x: normalize_text(x['parent_comment']),axis=1)
# df['cleaned_parent_comment'] = df['parent_comment'].swifter.apply(lambda x: normalize_text(x),axis=1)


Dask Apply: 100%|██████████| 32/32 [09:11<00:00, 17.22s/it]


Wall time: 9min 30s


In [20]:
df[['parent_comment','cleaned_parent_comment']].head()

,parent_comment,cleaned_parent_comment
0,"Yeah, I get that argument. At this point, I'd ...",yeah i get that argument at this point i prefe...
1,The blazers and Mavericks (The wests 5 and 6 s...,the blazers and mavericks the wests 5 and 6 se...
2,They're favored to win.,they favored to win
3,deadass don't kill my buzz,deadass do kill my buzz
4,Yep can confirm I saw the tool they use for th...,yep can confirm i saw the tool they use for th...


In [21]:
df.to_csv(r"C:\Users\prasr\Documents\Northeastern\IDMP\Project\reddit_sarcasm_detection\data\interim\parent_cleaned_sarcasm.csv")

## 1.2 Sentiment analysis on parent

In [11]:
!  pip install torch
import flair, torch
flair.device = torch.device('cpu') 

In [22]:
%%time
from flair.models import TextClassifier
from flair.data import Sentence
sia = TextClassifier.load('en-sentiment')
def flair_prediction(x):
    try:
        sentence = Sentence(x)
        sia.predict(sentence)
        # print(sentence.labels)
        score = sentence.labels[0]
        if "POSITIVE" in str(score):
            return "pos"
        elif "NEGATIVE" in str(score):
            return "neg"
        else:
            return "neu"
    except:
        return "prob"
# flair_prediction("20340i24i")
df["sentiment"] = df.swifter.apply(lambda x: flair_prediction(x["cleaned_parent_comment"]),axis=1)
# ddf['sentiment'] = ddf.apply(lambda x: flair_prediction(x['cleaned_parent_comment']),axis=1).compute()

2021-11-28 15:57:17,122 loading file C:\Users\prasr\.flair\models\sentiment-en-mix-distillbert_4.pt
Wall time: 2.12 s


'pos'

In [9]:
df = pd.read_csv(r'C:\Users\prasr\Documents\Northeastern\IDMP\Project\reddit_sarcasm_detection\data\interim\parent_cleaned_sarcasm_with_sentiment.csv')
df.head()
df = df.fillna('')

In [10]:
%%time
df['cleaned_comment'] = df.swifter.apply(lambda x: normalize_text(x['comment']),axis=1)
# df['cleaned_parent_comment'] = df['parent_comment'].swifter.apply(lambda x: normalize_text(x),axis=1)


Dask Apply: 100%|██████████| 32/32 [06:36<00:00, 12.40s/it]


Wall time: 6min 55s


In [12]:
%%time
from flair.models import TextClassifier
from flair.data import Sentence
sia = TextClassifier.load('en-sentiment')
def flair_prediction(x):
    try:
        sentence = Sentence(x)
        sia.predict(sentence)
        # print(sentence.labels)
        score = sentence.labels[0]
        if "POSITIVE" in str(score):
            return "pos"
        elif "NEGATIVE" in str(score):
            return "neg"
        else:
            return "neu"
    except:
        return "prob"
# flair_prediction("20340i24i")
df["sentiment"] = df.swifter.apply(lambda x: flair_prediction(x["cleaned_comment"]),axis=1)


2021-11-30 00:02:47,201 loading file C:\Users\prasr\.flair\models\sentiment-en-mix-distillbert_4.pt


Dask Apply:   6%|▋         | 2/32 [3:03:14<45:48:33, 5497.10s/it]


In [ ]:
from nltk.tokenize import TweetTokenizer
import re
def clean_text(comment):
    sent = str(comment)
    sent = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',' ',sent)
    sent = re.sub("<.*?>", " ", sent)
    sent = re.sub(r"[0-9]+"," ",sent)
    sent = re.sub(r"@[A-Za-z0-9]+"," ",sent)
    sent = re.sub(r"won't", "will not", sent)
    sent = re.sub(r"can\'t", "can not", sent)
    sent = re.sub(r"n\'t", " not", sent)
    sent = re.sub(r"\'re", " are", sent)
    sent = re.sub(r"\'s", " is", sent)
    sent = re.sub(r"\'d", " would", sent)
    sent = re.sub(r"\'ll", " will", sent)
    sent = re.sub(r"\'t", " not", sent)
    sent = re.sub(r"\'ve", " have", sent)
    sent = re.sub(r"\'m", " am", sent)
    sent = re.sub(r"n\'t", ' not',sent)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+',' ', sent)
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    sent = ' '.join(token for token in tokenizer.tokenize(sent.lower()) if token not in stops)
    sent = sent.lower().strip()
    return sent

df["cleaned_comment"] = df.swifter.apply(lambda x: clean_text(x["comment"]),axis=1)


In [25]:
import pandas as pd
df = pd.read_csv(r'C:\Users\prasr\Documents\Northeastern\IDMP\Project\reddit_sarcasm_detection\data\processed\performance.csv')
# df = df.sort_values(by=['Accuracy'])
df['Accuracy'] = df['Accuracy'].apply(lambda x: float(x.replace('%','')))
df.head()

,Model,Accuracy
0,Logistic Regression with TF-IDF,68.32
1,Decision Tree with TF-IDF,64.16
2,Random Forest with TF-IDF,67.76
3,Logistic Regression with GloVE,57.41
4,Decision Tree with GloVE,56.08


In [26]:
import plotly.express as px
import matplotlib.pyplot as plt
fig = px.bar(df, x='Model', y='Accuracy',height=600,width=800)
fig.update_yaxes(autorange = True)
fig.show()

In [27]:
df = pd.read_csv(r'C:\Users\prasr\Documents\Northeastern\IDMP\Project\reddit_sarcasm_detection\data\interim\parent_cleaned_sarcasm_with_sentiment.csv')
df.head()
df = df.fillna('')

In [28]:
!pip install emot

In [44]:
import emot ,swifter
emot_obj = emot.core.emot() 
text = ":)" 
emot_obj.emoticons(text)['flag']

True

In [45]:
df['has_emot'] = df['comment'].swifter.apply(lambda x: emot_obj.emoticons(x)['flag'])

Pandas Apply: 100%|██████████| 1004675/1004675 [00:03<00:00, 310892.19it/s]


In [46]:
df[df['comment'].apply(lambda x: ')' in x)]['comment']

5          I don't pay attention to her, but as long as s...
60         While I'm no fan of Bo1s (and I hate tournamen...
105        lets hope this 'Swindle' is a perpetual check ...
133                               No problem, thank *you* :)
135        And even wars such as Egyptian Unification war...
                                 ...                        
1004153                                   And the USA (Iraq)
1004228    Of course it's a "less of a country", those pe...
1004319    They should have other options, like: * Homeop...
1004472    Let's all hurry to see it while it's still ali...
1004589    So to confirm, no need to compile anything for...
Name: comment, Length: 20525, dtype: object

In [47]:
df['has_emot'].value_counts()

False    988305
True      16370
Name: has_emot, dtype: int64

In [54]:
import plotly.express as px
tmp = df.groupby(['has_emot']).size().reset_index(name='count').sort_values(by=['count'],ascending=False)
# tmp['label'] = tmp['label'].apply(lambda x: 'Sarcastic' if x==1 else 'Not Sarcastic')
# tmp.rename({has})
fig = px.bar(tmp, x='has_emot', y='count', color = 'has_emot',title= "Distribution of emoticons in sarcastic and non sarcastic comments")
fig.show()

In [55]:
import plotly.express as px
tmp = df.groupby(['has_emot','label']).size().reset_index(name='count').sort_values(by=['count'],ascending=False)
tmp['label'] = tmp['label'].apply(lambda x: 'Sarcastic' if x==1 else 'Not Sarcastic')
# tmp.rename({has})
tmp = tmp[tmp['label'] == 'Sarcastic']
fig = px.bar(tmp, x='has_emot', y='count', color = 'label',title= "Very few sarcastic comments have emoticons")
fig.show()